<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 20px; font-family: TimesNewRoman; color: skyblue">
    TIME SERIES DATA PROCESSING
    <br>
    LOAD REAL TIME | LOAD DAY AHEAD
<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color: skyblue">
    Main Formatting Notebook
</div>
</div>
<br>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
        Each part of the following script was used to proccess the raw data for the Real Time and Day Ahead Load Time Series Raw Data for all the european countries of the Dispa-SET_Unleash project.
<br>
Read explanation text cells to follow and understand all the process until final results were got stept by step.
</div>
<br>
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    1. Notebook Set Up
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Importing needed libraries
</div>

In [100]:
import pandas as pd
from entsoe import EntsoePandasClient
from entsoe.exceptions import NoMatchingDataError  # Ensure this line is included
from entsoe.exceptions import NoMatchingDataError, InvalidBusinessParameterError  # Import relevant exceptions
import os
import csv
from datetime import datetime
import pytz
from pytz import timezone, utc
import shutil

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    2. Dispa-SET_Unleash Folder Path
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Determinning dynamically the zone_folder_path based on the location of the "Dispa-SET_Unleash" folder relative to the current working directory.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
- If the "Dispa-SET_Unleash" folder is copied to a different machine or location, the dispaSET_unleash_folder_path variable will automatically adjust accordingly.
</div>

In [101]:
# Get the current working directory
current_directory = os.getcwd()

# Navigate to the parent directory of "Dispa-SET_Unleash"
dispaSET_unleash_parent_directory = os.path.dirname(current_directory)

# Get the path to the "Dispa-SET_Unleash" folder
dispaSET_unleash_folder_path = os.path.dirname(dispaSET_unleash_parent_directory)

# Construct the dispaSET_unleash_folder_name variable
dispaSET_unleash_folder_name = os.path.basename(dispaSET_unleash_folder_path)

print("dispaSET_unleash_folder_name:", dispaSET_unleash_folder_name)
print("dispaSET_unleash_folder_path:", dispaSET_unleash_folder_path)

dispaSET_unleash_folder_name: Dispa-SET_Unleash
dispaSET_unleash_folder_path: /home/ray/Dispa-SET_Unleash


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    3. Usefull Variable Definition
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Entering a value to all the variables which content are going to be used in some of the next stages of this script. 
</div>
<div style="text-align: Justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Indicate the year of all data is referring to in the variable data_year.
<br>
The universal_standar_time variable is going to be used to download all the time series data in this horary zone. Additionally as each european country belongs a particular time sector the corresponding time series data related to its time sector are going to be downloaded as well but in a different file.
<br>Additionally there are some default parameters that has to be defined to the correct working and calling to the ENTSO-E downloading functions.
</div>

In [102]:
# Year to which data refers to:
data_year = 2023
data_year_str = str(data_year)

In [109]:
# Additional string to be appended
additional_path = "/RawData/Load_RealTime/"
additional_path_1 = "/RawData/Load_DayAhead/"
additional_path_2 = "/RawData/Load_RealTime/Raw_Data_Sources/"

# Construct the Outage_Factors_folder_path variable
load_realtime_folder_path = dispaSET_unleash_folder_path + additional_path

# Construct the Outage_Factors_folder_path variable
load_dayAhead_folder_path = dispaSET_unleash_folder_path + additional_path_1

# Construct the Outage_Factors_Raw_Data_folder_path variable
load_raw_data_folder_path = dispaSET_unleash_folder_path + additional_path_2

print("load_realtime_folder_path:", load_realtime_folder_path)
print("load_dayAhead_folder_path:", load_dayAhead_folder_path)
print("load_raw_data_folder_path:", load_raw_data_folder_path)

load_realtime_folder_path: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/
load_dayAhead_folder_path: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/
load_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
    3.1. Back Up Directory
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Saving the original files into a Back up folder.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Since in the next steps of the processing data new subfolders and files are going to be created, the original ones are saved in a back up foldet to return them as its default content ones the process will be finished.
</div>

In [104]:
additional_path_5 = '/RawData/Load_RealTime_backup/'

# Construct the backup_folder_path variable
backup_folder_path = dispaSET_unleash_folder_path + additional_path_5

print("backup_folder_path:", backup_folder_path)

# Create a backup of the directory
if os.path.exists(backup_folder_path):
    shutil.rmtree(backup_folder_path)  # Remove any existing backup if necessary
shutil.copytree(load_realtime_folder_path, backup_folder_path)

print(f"Backup created at {backup_folder_path}")

backup_folder_path: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime_backup/
Backup created at /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime_backup/


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    4. Country List Variable Definition
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Defining the list of countries according to the available data. 
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Just those countries that interchange flows with other countries different of the ones modelled in Dispa-SET are defined in the list.
</div>

In [105]:
# Define a list of country codes
load_data_country_list = ["AT", "BE", "BG", "CH", "CY", "CZ", "DE", "DK", "EE", "GR", "ES", "FI", "FR", "HR", "HU", 
                          "IE", "IT", "LT", "LU", "LV", "MT", "NL", "NO", "PL", "PT", "RO", "SE", "SI", "SK", "UK"]

In [110]:
# Define the directory and file path
file_name = 'country_list.csv'
file_path = os.path.join(load_raw_data_folder_path, file_name)

# Ensure the directory exists
os.makedirs(load_raw_data_folder_path, exist_ok=True)

# Create a DataFrame
df = pd.DataFrame(load_data_country_list, columns=['Country_From'])

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

print(f"DataFrame saved to '{file_path}'")
load_data_country_list_file = file_path

DataFrame saved to '/home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/country_list.csv'


<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [113]:
print (f"dispaSET_unleash_folder_name:                              {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path:                              {dispaSET_unleash_folder_path}")
print (f"data_year:                                                 {data_year}")
print (f"load_realtime_folder_path:                                 {load_realtime_folder_path}")   
print (f"load_dayAhead_folder_path:                                 {load_dayAhead_folder_path}")
print (f"load_raw_data_folder_path:                                 {load_raw_data_folder_path}")
print (f"load_data_country_list_file:                               {load_data_country_list_file}")

dispaSET_unleash_folder_name:                              Dispa-SET_Unleash
dispaSET_unleash_folder_path:                              /home/ray/Dispa-SET_Unleash
data_year:                                                 2023
load_realtime_folder_path:                                 /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/
load_dayAhead_folder_path:                                 /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/
load_raw_data_folder_path:                                 /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/
load_data_country_list_file:                               /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/country_list.csv


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;color:skyblue">
Defining the path to the sub-folders where all the Load Real Time and Load Day Ahead raw data is saved. 
</div>

In [118]:

def create_country_folder_column(load_data_country_list_file, load_raw_data_folder_path, data_year_str):
  """
  Creates a Country_Folder column in the specified CSV file.

  Args:
    load_data_country_list_file: The path to the CSV file.
    load_raw_data_folder_path: The path to the load raw data folder.
    data_year: The data year.
  """

  df = pd.read_csv(load_data_country_list_file)

  def get_folder_path(country):
    return os.path.join(load_raw_data_folder_path, data_year_str, country)

  df['Country_Folder'] = df['Country_From'].apply(get_folder_path)
  df.to_csv(load_data_country_list_file, index=False)

  print("Country_Folder column created successfully.")

create_country_folder_column(load_data_country_list_file, load_raw_data_folder_path, data_year_str)

Country_Folder column created successfully.


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;color:skyblue">
Defining the path to the files where all the Load Real Time and Lead Day Ahead raw data is saved. 
</div>

In [120]:
def create_raw_data_file_path_column(load_data_country_list_file):
  """
  Creates a Raw_Data_File_Path column in the specified CSV file.

  Args:
    load_data_country_list_file: The path to the CSV file.
  """

  df = pd.read_csv(load_data_country_list_file)

  def get_raw_data_file_path(country_folder):
    for root, dirs, files in os.walk(country_folder):
      for file in files:
        if file.endswith('.csv'):
          return os.path.join(root, file)
    return None

  df['Raw_Data_File_Path'] = df['Country_Folder'].apply(get_raw_data_file_path)
  df.to_csv(load_data_country_list_file, index=False)

  print("Raw_Data_File_Path column created successfully.")

create_raw_data_file_path_column(load_data_country_list_file)

Raw_Data_File_Path column created successfully.


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Since the Acronym of Grece in the downloaded data is 'GR' and the Dispa-SET format for the country is 'EL'. All the needed changes in the used variables are done.
<br>

In [121]:
# Read the CSV file into a DataFrame
df = pd.read_csv(load_data_country_list_file)

# Replace 'GR' with 'EL' in the entire DataFrame
df = df.applymap(lambda x: x.replace('GR', 'EL') if isinstance(x, str) else x)

# Save the updated DataFrame back to the CSV file
df.to_csv(load_data_country_list_file, index=False)

print(f"Replacements made and file saved: {load_data_country_list_file}")

Replacements made and file saved: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/country_list.csv


/tmp/ipykernel_143036/2568165819.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('GR', 'EL') if isinstance(x, str) else x)


In [122]:
data_year_str = str(data_year)

load_raw_data_folder_path_1 = load_raw_data_folder_path + data_year_str

# Walk through the directory tree
for root, dirs, files in os.walk(load_raw_data_folder_path_1, topdown=False):  
    # Rename files
    for name in files:
        if 'GR' in name:
            new_name = name.replace('GR', 'EL')
            old_file_path = os.path.join(root, name)
            new_file_path = os.path.join(root, new_name)
            os.rename(old_file_path, new_file_path)
            print(f"Renamed file: {old_file_path} to {new_file_path}")

    # Rename directories
    for name in dirs:
        if 'GR' in name:
            new_name = name.replace('GR', 'EL')
            old_dir_path = os.path.join(root, name)
            new_dir_path = os.path.join(root, new_name)
            os.rename(old_dir_path, new_dir_path)
            print(f"Renamed directory: {old_dir_path} to {new_dir_path}")

Renamed directory: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2023/GR to /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2023/EL


In [123]:
# Function to replace 'GR' with 'EL'
def replace_gr_with_el(lst):
    return ['EL' if x == 'GR' else x for x in lst]

# Applying the function to both lists
load_data_country_list = replace_gr_with_el(load_data_country_list)

# Print the updated lists
print("Updated load_data_country_list:", load_data_country_list)

Updated load_data_country_list: ['AT', 'BE', 'BG', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'FI', 'FR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'NO', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK', 'UK']


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    5. Raw Data Formating
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Addapting the time step data to the UTC for all the countries.
</div>

In [124]:
# Read the country list CSV file
country_list_df = pd.read_csv(load_data_country_list_file)

# Ensure the 'Country_File_Path' column exists
if 'Raw_Data_File_Path' not in country_list_df.columns:
    raise ValueError("Column 'Raw_Data_File_Path' does not exist in the CSV file")

# Define the function to convert time to UTC
def convert_to_utc(time_str):
    local_time = datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S%z')
    utc_time = local_time.astimezone(pytz.utc)
    return utc_time.strftime('%Y-%m-%d %H:%M:%S%z')

# Process each CSV file
for file_path in country_list_df['Raw_Data_File_Path'].dropna():
    # Ensure the file exists
    if not os.path.isfile(file_path):
        print(f"File not found: {file_path}")
        continue

    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Check if the 'index' column exists
    if 'index' not in df.columns:
        print(f"'index' column not found in file: {file_path}")
        continue

    # Convert the 'index' column to UTC
    df['index'] = df['index'].apply(convert_to_utc)
    
    # Save the updated CSV file
    df.to_csv(file_path, index=False)
    print(f"Updated file saved: {file_path}")

Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2023/AT/2023_1.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2023/BE/2023_1.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2023/BG/2023_1.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2023/CH/2023_1.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2023/CY/2023_1.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2023/CZ/2023_1.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2023/DE/2023_1.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2023/DK/2023_1.csv
Updated file saved: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2023/EE/2023_1.csv
Updated file saved: /home/ray/Dispa-SET_Unleas

In [125]:
# Step 1: Read country list CSV file
country_list_df = pd.read_csv(load_data_country_list_file)

# Step 2-6: Process each CSV file specified in Raw_Data_File_Path column
for index, row in country_list_df.iterrows():
    raw_data_file_path = row['Raw_Data_File_Path']
    
    # Step 2: Read the original CSV file
    df = pd.read_csv(raw_data_file_path)
    df['index'] = pd.to_datetime(df['index'], format='%Y-%m-%d %H:%M:%S%z')
    df.sort_values(by='index', inplace=True)
    df['time_diff'] = df['index'].diff().dt.total_seconds()
    min_time_step = df['time_diff'].dropna().min()
    
    print(f"Processing file: {raw_data_file_path}")
    print(f"The minimum time step used in the column is: {min_time_step} seconds")
    
    # Step 3: Create a new DataFrame with reconstructed time values
    start_date = pd.Timestamp(f'{data_year}-01-01')
    end_date = pd.Timestamp(f'{data_year}-12-31 23:59:59')  # Assuming end of year, adjust as needed
    time_step = pd.Timedelta(seconds=min_time_step)
    date_range = pd.date_range(start=start_date, end=end_date, freq=f'{min_time_step}S')
    df_new = pd.DataFrame(date_range, columns=['index'])


    # Step 4: Verify and filter rows based on data_year in the original CSV file
    df_filtered = df[df['index'].dt.year == data_year]
    df_non_matching = df[df['index'].dt.year != data_year]
    df_filtered = pd.concat([df_filtered, df_non_matching], ignore_index=True)
    
    # Step 5: Copy 'Forecasted Load' and 'Actual Load' columns to the new CSV file
    df_to_write = df_new.copy()
    df_to_write['Forecasted Load'] = df_filtered['Forecasted Load']
    df_to_write['Actual Load'] = df_filtered['Actual Load']
    
    # Step 6: Save the final DataFrame to a new CSV file
    new_file_path = os.path.join(os.path.dirname(raw_data_file_path), f"{data_year}.csv")
    df_to_write.to_csv(new_file_path, index=False)
    print(f"Updated CSV file with 'Forecasted Load' and 'Actual Load' columns: {new_file_path}")
    
    # Update the corresponding Raw_Data_File_Path in country_list.csv with the new file path
    country_list_df.at[index, 'Raw_Data_File_Path'] = new_file_path

# Step 7: Save the updated country_list_df back to the CSV file
country_list_df.to_csv(load_data_country_list_file, index=False)
print(f"Updated country_list.csv with new file paths.")

Processing file: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2023/AT/2023_1.csv
The minimum time step used in the column is: 900.0 seconds


/tmp/ipykernel_143036/2315473137.py:22: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  date_range = pd.date_range(start=start_date, end=end_date, freq=f'{min_time_step}S')


Updated CSV file with 'Forecasted Load' and 'Actual Load' columns: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2023/AT/2023.csv


/tmp/ipykernel_143036/2315473137.py:22: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  date_range = pd.date_range(start=start_date, end=end_date, freq=f'{min_time_step}S')


Processing file: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2023/BE/2023_1.csv
The minimum time step used in the column is: 900.0 seconds
Updated CSV file with 'Forecasted Load' and 'Actual Load' columns: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2023/BE/2023.csv
Processing file: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2023/BG/2023_1.csv
The minimum time step used in the column is: 3600.0 seconds


...

Updated CSV file with 'Forecasted Load' and 'Actual Load' columns: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2023/SI/2023.csv
Processing file: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2023/SK/2023_1.csv
The minimum time step used in the column is: 3600.0 seconds


/tmp/ipykernel_143036/2315473137.py:22: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  date_range = pd.date_range(start=start_date, end=end_date, freq=f'{min_time_step}S')


Updated CSV file with 'Forecasted Load' and 'Actual Load' columns: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2023/SK/2023.csv
Processing file: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2023/UK/2023_1.csv
The minimum time step used in the column is: 1800.0 seconds


/tmp/ipykernel_143036/2315473137.py:22: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  date_range = pd.date_range(start=start_date, end=end_date, freq=f'{min_time_step}S')


Updated CSV file with 'Forecasted Load' and 'Actual Load' columns: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/2023/UK/2023.csv
Updated country_list.csv with new file paths.


In [128]:
# Load the list of file paths from the specified CSV file
country_list_df = pd.read_csv(load_data_country_list_file)

# Function to process each file
def process_csv_file(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)

    # Ensure the 'index' column exists and modify it by appending '+00:00'
    if 'index' in df.columns:
        df['index'] = df['index'].astype(str) + '+00:00'

    # Save the modified DataFrame back to the original CSV file
    df.to_csv(file_path, index=False)

# Iterate over each file path in the 'Raw_Data_File_Path' column
for file_path in country_list_df['Raw_Data_File_Path']:
    process_csv_file(file_path)

print("Processing complete. All 'index' columns have been updated with '+00:00'.")


Processing complete. All 'index' columns have been updated with '+00:00'.


</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Dividing the day Ahead and Real Time load data into diferent files.
</div>

In [129]:
# Load the country list file
print("Loading the country list file...")
country_list_df = pd.read_csv(load_data_country_list_file)

# Initialize new columns for the output paths
country_list_df['Load_DayAhead_File_Path'] = ''
country_list_df['Load_Realtime_File_Path'] = ''

# Process each country
for index, row in country_list_df.iterrows():
    raw_data_file_path = row['Raw_Data_File_Path']
    country_from = row['Country_From']
    
    print(f"Processing {country_from}...")

    # Check if the raw data file exists
    if not os.path.exists(raw_data_file_path):
        print(f"Error: File not found - {raw_data_file_path}")
        continue
    
    # Load the raw data CSV file
    raw_data_df = pd.read_csv(raw_data_file_path)
    
    # Create DataFrames for Forecasted Load and Actual Load
    day_ahead_df = raw_data_df[['index', 'Forecasted Load']]
    real_time_df = raw_data_df[['index', 'Actual Load']]
    
    # Create subdirectories for the country if they don't exist
    country_dayAhead_path = os.path.join(load_dayAhead_folder_path, country_from)
    country_realTime_path = os.path.join(load_realtime_folder_path, country_from)
    
    if not os.path.exists(country_dayAhead_path):
        print(f"Creating directory: {country_dayAhead_path}")
        os.makedirs(country_dayAhead_path, exist_ok=True)
    
    if not os.path.exists(country_realTime_path):
        print(f"Creating directory: {country_realTime_path}")
        os.makedirs(country_realTime_path, exist_ok=True)
    
    # Define paths for the new CSV files with the year as the file name
    day_ahead_file_path = os.path.join(country_dayAhead_path, f"{data_year}.csv")
    real_time_file_path = os.path.join(country_realTime_path, f"{data_year}.csv")
    
    # Save the new CSV files
    day_ahead_df.to_csv(day_ahead_file_path, index=False)
    print(f" - Saved day-ahead data to {day_ahead_file_path}")
    
    real_time_df.to_csv(real_time_file_path, index=False)
    print(f" - Saved real-time data to {real_time_file_path}")
    
    # Update the paths in the country list DataFrame
    country_list_df.at[index, 'Load_DayAhead_File_Path'] = day_ahead_file_path
    country_list_df.at[index, 'Load_Realtime_File_Path'] = real_time_file_path

# Save the updated country list DataFrame
print("Saving the updated country list file...")
country_list_df.to_csv(load_data_country_list_file, index=False)

print("Process completed successfully!")

Loading the country list file...
Processing AT...
Creating directory: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/AT
Creating directory: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/AT
 - Saved day-ahead data to /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/AT/2023.csv
 - Saved real-time data to /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/AT/2023.csv
Processing BE...
Creating directory: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BE
Creating directory: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/BE
 - Saved day-ahead data to /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BE/2023.csv
 - Saved real-time data to /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/BE/2023.csv
Processing BG...
Creating directory: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BG
Creating directory: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/BG
 - Saved day-ahead data to /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BG/2023.csv
 - Saved real-time data to /home/

In [130]:
# Load the country list file
print(f"Loading country list file from {load_data_country_list_file}...")
country_list_df = pd.read_csv(load_data_country_list_file)

# Define the new directory names for each time step
intervals = ['1h', '30min', '15min']
time_steps = {
    '1h': '1h',
    '30min': '30T',
    '15min': '15T'
}

# Function to extract rows at a specific time step and save to a new CSV file
def extract_and_save(file_path, interval, folder_name):
    # Extract the base directory from the file path
    base_directory = os.path.dirname(file_path)
    
    # Create the new directory for the time step
    new_dir = os.path.join(base_directory, folder_name)
    os.makedirs(new_dir, exist_ok=True)

    # Read the original CSV file
    print(f"Loading CSV file from {file_path}...")
    df = pd.read_csv(file_path)
    
    # Ensure the first column is named 'index'
    if df.columns[0] != 'index':
        df.rename(columns={df.columns[0]: 'index'}, inplace=True)

    # Convert the 'index' column to datetime
    df['index'] = pd.to_datetime(df['index'], format='%Y-%m-%d %H:%M:%S%z')

    # Set the 'index' column as the DataFrame index
    df_resample = df.set_index('index')

    # Resample the DataFrame based on the interval
    resampled_df = df_resample.resample(interval).first().reset_index()

    # Define the new file path
    new_file_path = os.path.join(new_dir, os.path.basename(file_path))
    
    # Save the resampled DataFrame to the new CSV file
    resampled_df.to_csv(new_file_path, index=False)
    print(f"File saved: {new_file_path}")

# Process each file in the 'Load_DayAhead_File_Path' and 'Load_Realtime_File_Path' columns
for index, row in country_list_df.iterrows():
    day_ahead_path = row['Load_DayAhead_File_Path']
    real_time_path = row['Load_Realtime_File_Path']

    print(f"Processing files for row {index + 1}...")
    
    # Apply the resampling and saving process for the day-ahead file
    if os.path.exists(day_ahead_path):
        print(f"Processing Day-Ahead file: {day_ahead_path}")
        for folder, interval in time_steps.items():
            extract_and_save(day_ahead_path, interval, folder)
    else:
        print(f"Error: Day-Ahead file not found at {day_ahead_path}")
    
    # Apply the resampling and saving process for the real-time file
    if os.path.exists(real_time_path):
        print(f"Processing Real-Time file: {real_time_path}")
        for folder, interval in time_steps.items():
            extract_and_save(real_time_path, interval, folder)
    else:
        print(f"Error: Real-Time file not found at {real_time_path}")

print("All files processed successfully!")


Loading country list file from /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/Raw_Data_Sources/country_list.csv...
Processing files for row 1...
Processing Day-Ahead file: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/AT/2023.csv
Loading CSV file from /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/AT/2023.csv...
File saved: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/AT/1h/2023.csv
Loading CSV file from /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/AT/2023.csv...
File saved: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/AT/30min/2023.csv
Loading CSV file from /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/AT/2023.csv...


/tmp/ipykernel_143036/3717144850.py:37: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = df_resample.resample(interval).first().reset_index()
/tmp/ipykernel_143036/3717144850.py:37: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = df_resample.resample(interval).first().reset_index()


File saved: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/AT/15min/2023.csv
Processing Real-Time file: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/AT/2023.csv
Loading CSV file from /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/AT/2023.csv...
File saved: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/AT/1h/2023.csv
Loading CSV file from /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/AT/2023.csv...


/tmp/ipykernel_143036/3717144850.py:37: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = df_resample.resample(interval).first().reset_index()


File saved: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/AT/30min/2023.csv
Loading CSV file from /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/AT/2023.csv...


...

File saved: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/UK/15min/2023.csv
Processing Real-Time file: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/UK/2023.csv
Loading CSV file from /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/UK/2023.csv...
File saved: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/UK/1h/2023.csv
Loading CSV file from /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/UK/2023.csv...


/tmp/ipykernel_143036/3717144850.py:37: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = df_resample.resample(interval).first().reset_index()


File saved: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/UK/30min/2023.csv
Loading CSV file from /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/UK/2023.csv...


/tmp/ipykernel_143036/3717144850.py:37: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = df_resample.resample(interval).first().reset_index()


File saved: /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime/UK/15min/2023.csv
All files processed successfully!


<div style="text-align: Justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Dividing the clean data in time stepts of 15 minutes, 30 minutes, and 1 hour.
</div>

In [131]:
# Load the list of file paths from the specified CSV file
country_list_df = pd.read_csv(load_data_country_list_file)

# Folders to check within each directory
time_intervals = ['1h', '15min', '30min']

# Function to check for missing values and delete folders if necessary
def check_and_delete(folder_path, data_year):
    try:
        # Construct the path to the CSV file within the folder
        csv_file_path = os.path.join(folder_path, f'{data_year}.csv')
        
        if os.path.exists(csv_file_path):
            # Read the CSV file
            df = pd.read_csv(csv_file_path)

            # Check if there are missing values in the second column
            second_column = df.columns[1]  # Get the name of the second column
            if df[second_column].isna().any():
                # If there are missing values, delete the folder and its content
                shutil.rmtree(folder_path)
                print(f"Deleted folder due to missing values: {folder_path}")
            else:
                print(f"No missing values in file: {csv_file_path}")
        else:
            print(f"CSV file not found: {csv_file_path}")
    except Exception as e:
        print(f"Error processing folder: {folder_path}. Error: {e}")

# Iterate over each path in the 'Load_DayAhead_File_Path' and 'Load_Realtime_File_Path' columns
for path_column in ['Load_DayAhead_File_Path', 'Load_Realtime_File_Path']:
    for file_path in country_list_df[path_column].dropna():  # Drop any NaN entries to avoid processing empty paths
        directory = os.path.dirname(file_path)  # Get the directory of the file

        for interval in time_intervals:
            interval_folder_path = os.path.join(directory, interval)
            if os.path.isdir(interval_folder_path):
                check_and_delete(interval_folder_path, data_year)

print("Processing complete. All specified folders have been checked and cleaned if necessary.")

No missing values in file: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/AT/1h/2023.csv
No missing values in file: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/AT/15min/2023.csv
No missing values in file: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/AT/30min/2023.csv
No missing values in file: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BE/1h/2023.csv
No missing values in file: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BE/15min/2023.csv
No missing values in file: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BE/30min/2023.csv
No missing values in file: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BG/1h/2023.csv
Deleted folder due to missing values: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BG/15min
Deleted folder due to missing values: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BG/30min
No missing values in file: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/CH/1h/2023.csv
Deleted folder due to missing values: /home/ray/Dispa-SET_

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Erasing all those unnecesary folders and unnecesary rows in the files.
</div>

In [132]:
# Time intervals in seconds
time_intervals = {
    '1h': 3600,
    '15min': 900,
    '30min': 1800
}

# Function to identify the minimum time step in the 'index' column
def identify_time_step(csv_file_path):
    df = pd.read_csv(csv_file_path)

    if 'index' not in df.columns:
        raise ValueError(f"'index' column not found in the file: {csv_file_path}")
    
    # Convert 'index' column to datetime
    df['index'] = pd.to_datetime(df['index'])
    
    # Calculate the differences between consecutive timestamps in seconds
    time_deltas = df['index'].diff().dropna().dt.total_seconds()
    
    # Identify the minimum time step used
    min_time_step = time_deltas.min()
    print(f"The minimum time step used in {csv_file_path} is: {min_time_step} seconds")
    return min_time_step

# Function to delete folders with smaller time steps
def delete_smaller_time_step_folders(base_directory, identified_time_step):
    for interval_name, interval_seconds in time_intervals.items():
        interval_folder_path = os.path.join(base_directory, interval_name)
        if os.path.isdir(interval_folder_path) and interval_seconds < identified_time_step:
            shutil.rmtree(interval_folder_path)
            print(f"Deleted folder: {interval_folder_path}")
        else:
            print(f"Kept folder: {interval_folder_path}")

# Load the list of file paths from the specified CSV file
country_list_df = pd.read_csv(load_data_country_list_file)

# Iterate over each path in the 'Load_DayAhead_File_Path' and 'Load_Realtime_File_Path' columns
for path_column in ['Load_DayAhead_File_Path', 'Load_Realtime_File_Path']:
    for file_path in country_list_df[path_column].dropna():  # Drop any NaN entries to avoid processing empty paths
        directory = os.path.dirname(file_path)  # Get the directory of the file
        
        # Construct the path to the CSV file named as the data_year
        year_csv_path = os.path.join(directory, f'{data_year}.csv')
        
        if os.path.exists(year_csv_path):
            identified_time_step = identify_time_step(year_csv_path)
            delete_smaller_time_step_folders(directory, identified_time_step)
        else:
            print(f"CSV file for the year {data_year} not found: {year_csv_path}")

print("Processing complete. All specified folders have been checked and cleaned if necessary.")

The minimum time step used in /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/AT/2023.csv is: 900.0 seconds
Kept folder: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/AT/1h
Kept folder: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/AT/15min
Kept folder: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/AT/30min
The minimum time step used in /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BE/2023.csv is: 900.0 seconds
Kept folder: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BE/1h
Kept folder: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BE/15min
Kept folder: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BE/30min
The minimum time step used in /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BG/2023.csv is: 3600.0 seconds
Kept folder: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BG/1h
Kept folder: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BG/15min
Kept folder: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BG/30min
The minimum time step used in 

In [133]:
# Load the list of file paths from the specified CSV file
country_list_df = pd.read_csv(load_data_country_list_file)

# Folders to check
folders_to_check = ['1h', '15min', '30min']

# Function to remove the first row (header) from a CSV file
def remove_csv_header(file_path):
    # Read the CSV file, skip the header, and save it back without the header
    df = pd.read_csv(file_path, skiprows=1)  # Skip the header row
    df.to_csv(file_path, index=False)  # Save without the original header
    print(f"Header removed from file: {file_path}")

# Iterate over each path in the 'Load_DayAhead_File_Path' and 'Load_Realtime_File_Path' columns
for path_column in ['Load_DayAhead_File_Path', 'Load_Realtime_File_Path']:
    for file_path in country_list_df[path_column].dropna():  # Drop any NaN entries to avoid processing empty paths
        directory = os.path.dirname(file_path)  # Get the directory of the file
        
        # Check each of the specified folders ('1h', '15min', '30min')
        for folder in folders_to_check:
            folder_path = os.path.join(directory, folder)
            
            if os.path.exists(folder_path) and os.path.isdir(folder_path):
                # Construct the path to the CSV file named as the data_year
                year_csv_path = os.path.join(folder_path, f'{data_year}.csv')
                
                if os.path.exists(year_csv_path):
                    remove_csv_header(year_csv_path)
                else:
                    print(f"CSV file not found: {year_csv_path}")

print("Processing complete. Headers removed where applicable.")


Header removed from file: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/AT/1h/2023.csv
Header removed from file: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/AT/15min/2023.csv
Header removed from file: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/AT/30min/2023.csv
Header removed from file: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BE/1h/2023.csv
Header removed from file: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BE/15min/2023.csv
Header removed from file: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BE/30min/2023.csv
Header removed from file: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/BG/1h/2023.csv
Header removed from file: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/CH/1h/2023.csv
Header removed from file: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/CZ/1h/2023.csv
Header removed from file: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/DE/1h/2023.csv
Header removed from file: /home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/DE

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Copying the time already formated Load Real Time and Day Ahead Load data to the main Dispa-SET data base dirtectory
</div>

In [134]:
additional_path_3 = "/Database/Load_RealTime/"
additional_path_4 = "/Database/Load_DayAhead/"

# Construct the power_plants_raw_data_folder_path variable
load_realTime_data_base_folder_path = dispaSET_unleash_folder_path + additional_path_3
load_dayAhead_data_base_folder_path = dispaSET_unleash_folder_path + additional_path_4

In [135]:
# Read the country list CSV file
country_list_df = pd.read_csv(load_data_country_list_file)

# Extract the 'Country_From' column values
countries = country_list_df['Country_From'].tolist()

# Function to copy files while preserving the existing structure
def copy_csv_files(src_base_path, dest_base_path, subfolders):
    for country in subfolders:
        src_country_folder = os.path.join(src_base_path, country)
        
        if os.path.exists(src_country_folder):
            for subfolder in os.listdir(src_country_folder):
                src_subfolder_path = os.path.join(src_country_folder, subfolder)
                
                if os.path.isdir(src_subfolder_path):
                    # Source file path
                    src_file_path = os.path.join(src_subfolder_path, f"{data_year}.csv")
                    
                    if os.path.exists(src_file_path):
                        # Destination path should mimic the structure
                        dest_country_folder = os.path.join(dest_base_path, country)
                        dest_subfolder_path = os.path.join(dest_country_folder, subfolder)
                        
                        # Ensure the destination subfolder exists
                        os.makedirs(dest_subfolder_path, exist_ok=True)
                        
                        # Destination file path
                        dest_file_path = os.path.join(dest_subfolder_path, f"{data_year}.csv")
                        
                        # Copy the file if it doesn't already exist in the destination
                        if not os.path.exists(dest_file_path):
                            shutil.copy2(src_file_path, dest_file_path)
                            print(f"Copied {src_file_path} to {dest_file_path}")
                        else:
                            print(f"File {dest_file_path} already exists. Skipping copy.")
        else:
            print(f"Source folder {src_country_folder} does not exist. Skipping.")

# Copy CSV files from Load_RealTime folder
copy_csv_files(load_realtime_folder_path, load_realTime_data_base_folder_path, countries)

# Copy CSV files from Load_DayAhead folder
copy_csv_files(load_dayAhead_folder_path, load_dayAhead_data_base_folder_path, countries)

print("All specified CSV files have been copied.")


File /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/AT/1h/2023.csv already exists. Skipping copy.
File /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/AT/30min/2023.csv already exists. Skipping copy.
File /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/AT/15min/2023.csv already exists. Skipping copy.
File /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/BE/1h/2023.csv already exists. Skipping copy.
File /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/BE/30min/2023.csv already exists. Skipping copy.
File /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/BE/15min/2023.csv already exists. Skipping copy.
File /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/CH/1h/2023.csv already exists. Skipping copy.
File /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/DE/1h/2023.csv already exists. Skipping copy.
File /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/DE/30min/2023.csv already exists. Skipping copy.
File /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/DE/15min/20

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
8.  Load Real Time and Load Day Ahead Folder Back Up
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Once all the formating process was done the Net Tranfer Capacity Folder is restored to its defoult state.
</div>

In [136]:
if os.path.exists(load_realtime_folder_path):
    shutil.rmtree(load_realtime_folder_path)  # Remove the current directory
shutil.copytree(backup_folder_path, load_realtime_folder_path)

print(f"Directory restored to original state from {backup_folder_path}")

Directory restored to original state from /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime_backup/


In [137]:
shutil.rmtree(backup_folder_path)
print(f"Backup folder {backup_folder_path} deleted successfully.")

Backup folder /home/ray/Dispa-SET_Unleash/RawData/Load_RealTime_backup/ deleted successfully.


In [138]:
# Iterate over all items in the directory
for item in os.listdir(load_dayAhead_folder_path):
    item_path = os.path.join(load_dayAhead_folder_path, item)
    
    # Check if the item is a directory
    if os.path.isdir(item_path):
        # Remove the directory and all its contents
        shutil.rmtree(item_path)

print(f"All folders and their contents inside '{load_dayAhead_folder_path}' have been deleted.")

All folders and their contents inside '/home/ray/Dispa-SET_Unleash/RawData/Load_DayAhead/' have been deleted.
